In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
gcc6_3_32 = pd.read_csv("../data/binutils_gcc3~9_op0~4_csv/"+'gcc6'+"_3_32.csv", index_col=0)

# 형태 출력
print(gcc6_3_32.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_3_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
gcc6_3_32.head()

(2329146, 2)
reset_index 완료
input data shape


,hex,bin
0,55,1
1,57,0
2,56,0
3,53,0
4,e8,0


In [2]:
gcc6_3_32.tail()

,hex,bin
2329141,c3,0
2329142,8b,1
2329143,1c,0
2329144,24,0
2329145,c3,0


In [3]:
# (2-1) 데이터체크 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc6_3_32['hex'].unique()))

# (3) get_dummies 변환 

# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
gcc6_3_32_onehot = pd.get_dummies(gcc6_3_32)


print('원핫인코딩완료')

print(gcc6_3_32_onehot.shape)

# (4) 데이터 체크 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc6_3_32_onehot['bin'].value_counts())

256
원핫인코딩완료
(2329146, 257)
0    2317594
1      11552
Name: bin, dtype: int64


In [4]:
gcc6_3_32_onehot.tail(10)

,bin,hex_00,hex_01,hex_02,hex_03,hex_04,hex_05,hex_06,hex_07,hex_08,...,hex_f6,hex_f7,hex_f8,hex_f9,hex_fa,hex_fb,hex_fc,hex_fd,hex_fe,hex_ff
2329136,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2329137,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2329138,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2329139,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2329140,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2329141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2329142,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2329143,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2329144,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2329145,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# (5-1) gcc3 6gram
######################## 
idx3 = gcc6_3_32_onehot[gcc6_3_32_onehot['bin']==1].index  # 407, 474 ...
ls3 = list(idx3)

# 최종 뽑을 행에 대한 index
ls_idx3 = [] 
left_idx3, right_idx3 = 0, 7 # 3개씩

# 6gram
for k in range(left_idx3, right_idx3):
    ls_idx3.extend(list(idx3 + k)) # index 형이라서 가능
print(len(ls_idx3))

#ls_idx3 = list(set(ls_idx3)) 
print(len(ls_idx3))

ls_idx3.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx3 = list(filter(lambda x: x<len(gcc6_3_32_onehot), ls_idx3))
print(len(ls_idx3))

# 2차 남은 index들 중 right_idx3 나눈 나머지 없애기
sub = len(ls_idx3)%(right_idx3)
print(sub)

ls_idx3 = ls_idx3[:len(ls_idx3)-sub]
print(len(ls_idx3))

print('gcc6_3_32', len(ls_idx3))

# loc 로 수정필요
gcc6_3_32_onehot_3gram = gcc6_3_32_onehot.loc[ls_idx3,:].copy()

80864
80864
80861
4
80857
gcc6_3_32 80857


In [6]:
# 훈련 데이터, 훈련 라벨
x_gcc6_3_32_3 = gcc6_3_32_onehot_3gram.iloc[:,1:].to_numpy()
y_gcc6_3_32_3 = gcc6_3_32_onehot_3gram['bin'].to_numpy()
print(x_gcc6_3_32_3.shape, y_gcc6_3_32_3.shape)


(80857, 256) (80857,)


In [7]:
x_gcc6_3_32_3 = x_gcc6_3_32_3.reshape(-1, right_idx3, x_gcc6_3_32_3.shape[1])
y_gcc6_3_32_3 = y_gcc6_3_32_3.reshape(-1, right_idx3, 1)

print(x_gcc6_3_32_3.shape, y_gcc6_3_32_3.shape)

(11551, 7, 256) (11551, 7, 1)


## 모델

In [8]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = Input(batch_shape=(None,right_idx3, 256)) 
xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

model1 = Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

Using TensorFlow backend.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 7, 256)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 7, 32)             34944     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 7, 1)              33        
Total params: 34,977
Trainable params: 34,977
Non-trainable params: 0
_________________________________________________________________


In [9]:
########## 3gram

# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_gcc6_3_32_3, y_gcc6_3_32_3):
    print('======Training stage======')
    model1.fit(x_gcc6_3_32_3[train],
               y_gcc6_3_32_3[train],
               epochs = 30,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_gcc6_3_32_3[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_gcc6_3_32_3[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

======Training stage======
Epoch 1/30
10395/10395 [==============================] - 2s 209us/step - loss: 0.2051 - accuracy: 0.9261
Epoch 2/30
10395/10395 [==============================] - 2s 152us/step - loss: 0.0345 - accuracy: 0.9890
Epoch 3/30
10395/10395 [==============================] - 2s 153us/step - loss: 0.0244 - accuracy: 0.9934
Epoch 4/30
10395/10395 [==============================] - 2s 152us/step - loss: 0.0178 - accuracy: 0.9955
Epoch 5/30
10395/10395 [==============================] - 2s 151us/step - loss: 0.0136 - accuracy: 0.9966
Epoch 6/30
10395/10395 [==============================] - 2s 153us/step - loss: 0.0118 - accuracy: 0.9970
Epoch 7/30
10395/10395 [==============================] - 2s 152us/step - loss: 0.0105 - accuracy: 0.9973
Epoch 8/30
10395/10395 [==============================] - 2s 152us/step - loss: 0.0096 - accuracy: 0.9975
Epoch 9/30
10395/10395 [==============================] - 2s 152us/step - loss: 0.0089 - accuracy: 0.9976
Epoch 10/30
10395/1

accuracy_score 0.9922145328719724
recall_score 0.9648
precision_score 0.9844897959183674
f1_score 0.9745454545454545
======Training stage======
Epoch 1/30
10396/10396 [==============================] - 2s 156us/step - loss: 0.0066 - accuracy: 0.9982
Epoch 2/30
10396/10396 [==============================] - 2s 159us/step - loss: 0.0060 - accuracy: 0.9984
Epoch 3/30
10396/10396 [==============================] - 2s 153us/step - loss: 0.0056 - accuracy: 0.9987
Epoch 4/30
10396/10396 [==============================] - 2s 181us/step - loss: 0.0053 - accuracy: 0.9987
Epoch 5/30
10396/10396 [==============================] - 2s 184us/step - loss: 0.0050 - accuracy: 0.9987
Epoch 6/30
10396/10396 [==============================] - 2s 183us/step - loss: 0.0048 - accuracy: 0.9988
Epoch 7/30
10396/10396 [==============================] - 2s 181us/step - loss: 0.0046 - accuracy: 0.9990
Epoch 8/30
10396/10396 [==============================] - 2s 181us/step - loss: 0.0044 - accuracy: 0.9989
Epoch 9/

accuracy_score 0.9956709956709957
recall_score 0.9807370184254607
precision_score 0.9898562975486053
f1_score 0.985275557425326
======Training stage======
Epoch 1/30
10396/10396 [==============================] - 3s 254us/step - loss: 0.0035 - accuracy: 0.9992
Epoch 2/30
10396/10396 [==============================] - 3s 245us/step - loss: 0.0029 - accuracy: 0.9994
Epoch 3/30
10396/10396 [==============================] - 3s 248us/step - loss: 0.0026 - accuracy: 0.9995
Epoch 4/30
10396/10396 [==============================] - 3s 252us/step - loss: 0.0024 - accuracy: 0.9995
Epoch 5/30
10396/10396 [==============================] - 3s 249us/step - loss: 0.0021 - accuracy: 0.9996
Epoch 6/30
10396/10396 [==============================] - 3s 252us/step - loss: 0.0019 - accuracy: 0.9996
Epoch 7/30
10396/10396 [==============================] - 3s 257us/step - loss: 0.0017 - accuracy: 0.9997
Epoch 8/30
10396/10396 [==============================] - 3s 293us/step - loss: 0.0018 - accuracy: 0.99

accuracy_score 0.9965367965367965
recall_score 0.9913928012519562
precision_score 0.9867601246105919
f1_score 0.9890710382513661
======Training stage======
Epoch 1/30
10396/10396 [==============================] - 4s 347us/step - loss: 0.0025 - accuracy: 0.99940s - loss: 0.0024 - accuracy - ETA: 0s - loss: 0.0024 - accu
Epoch 2/30
10396/10396 [==============================] - 4s 349us/step - loss: 0.0022 - accuracy: 0.9995
Epoch 3/30
10396/10396 [==============================] - 4s 340us/step - loss: 0.0019 - accuracy: 0.9995
Epoch 4/30
10396/10396 [==============================] - 4s 356us/step - loss: 0.0017 - accuracy: 0.9996
Epoch 5/30
10396/10396 [==============================] - 4s 348us/step - loss: 0.0018 - accuracy: 0.9996
Epoch 6/30
10396/10396 [==============================] - 4s 350us/step - loss: 0.0017 - accuracy: 0.9996
Epoch 7/30
10396/10396 [==============================] - 4s 350us/step - loss: 0.0016 - accuracy: 0.9996
Epoch 8/30
10396/10396 [==================

accuracy_score 0.9992578849721707
recall_score 0.9974662162162162
precision_score 0.9974662162162162
f1_score 0.9974662162162162
======Training stage======
Epoch 1/30
10396/10396 [==============================] - 4s 340us/step - loss: 0.0011 - accuracy: 0.9997
Epoch 2/30
10396/10396 [==============================] - 4s 362us/step - loss: 0.0010 - accuracy: 0.9997
Epoch 3/30
10396/10396 [==============================] - 4s 367us/step - loss: 9.9951e-04 - accuracy: 0.9998
Epoch 4/30
10396/10396 [==============================] - 4s 354us/step - loss: 9.5515e-04 - accuracy: 0.9998
Epoch 5/30
10396/10396 [==============================] - 4s 341us/step - loss: 9.2500e-04 - accuracy: 0.9998
Epoch 6/30
10396/10396 [==============================] - 3s 334us/step - loss: 9.5807e-04 - accuracy: 0.9998
Epoch 7/30
10396/10396 [==============================] - 3s 287us/step - loss: 9.6666e-04 - accuracy: 0.9998
Epoch 8/30
10396/10396 [==============================] - 3s 281us/step - loss: 8.

...

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
======Training stage======
Epoch 1/30
10396/10396 [==============================] - 2s 230us/step - loss: 6.0190e-04 - accuracy: 0.9999
Epoch 2/30
10396/10396 [==============================] - 2s 232us/step - loss: 5.2757e-04 - accuracy: 0.9999
Epoch 3/30
10396/10396 [==============================] - 2s 234us/step - loss: 5.5803e-04 - accuracy: 0.9999
Epoch 4/30
10396/10396 [==============================] - 2s 232us/step - loss: 5.0863e-04 - accuracy: 0.9999
Epoch 5/30
10396/10396 [==============================] - 2s 232us/step - loss: 5.6580e-04 - accuracy: 0.9999
Epoch 6/30
10396/10396 [==============================] - 2s 234us/step - loss: 5.0589e-04 - accuracy: 0.9999
Epoch 7/30
10396/10396 [==============================] - 2s 231us/step - loss: 5.3466e-04 - accuracy: 0.9999
Epoch 8/30
10396/10396 [==============================] - 2s 228us/step - loss: 5.1053e-04 - accuracy: 0.9999
Epoch 9/30
10396/10396 [

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
======Training stage======
Epoch 1/30
10396/10396 [==============================] - 2s 229us/step - loss: 2.7773e-04 - accuracy: 0.9999
Epoch 2/30
10396/10396 [==============================] - 2s 229us/step - loss: 2.3010e-04 - accuracy: 0.9999
Epoch 3/30
10396/10396 [==============================] - 2s 233us/step - loss: 2.4707e-04 - accuracy: 0.9999
Epoch 4/30
10396/10396 [==============================] - 2s 230us/step - loss: 2.7471e-04 - accuracy: 0.9999
Epoch 5/30
10396/10396 [==============================] - 2s 233us/step - loss: 2.1076e-04 - accuracy: 0.9999
Epoch 6/30
10396/10396 [==============================] - 2s 230us/step - loss: 2.5224e-04 - accuracy: 0.9999
Epoch 7/30
10396/10396 [==============================] - 2s 234us/step - loss: 2.6142e-04 - accuracy: 1.0000
Epoch 8/30
10396/10396 [==============================] - 2s 235us/step - loss: 2.3358e-04 - accuracy: 0.9999
Epoch 9/30
10396/10396 [

accuracy_score 0.9996289424860854
recall_score 0.9974048442906575
precision_score 1.0
f1_score 0.9987007362494587
======Training stage======
Epoch 1/30
10396/10396 [==============================] - ETA: 0s - loss: 4.9649e-04 - accuracy: 0.99 - 2s 231us/step - loss: 4.9515e-04 - accuracy: 0.9999
Epoch 2/30
10396/10396 [==============================] - 2s 230us/step - loss: 3.2826e-04 - accuracy: 1.0000
Epoch 3/30
10396/10396 [==============================] - 2s 234us/step - loss: 3.4014e-04 - accuracy: 1.0000
Epoch 4/30
10396/10396 [==============================] - 2s 233us/step - loss: 3.1660e-04 - accuracy: 1.0000
Epoch 5/30
10396/10396 [==============================] - 2s 233us/step - loss: 3.3103e-04 - accuracy: 1.0000
Epoch 6/30
10396/10396 [==============================] - 2s 231us/step - loss: 3.3681e-04 - accuracy: 1.0000
Epoch 7/30
10396/10396 [==============================] - 2s 230us/step - loss: 3.4268e-04 - accuracy: 1.0000
Epoch 8/30
10396/10396 [===================

accuracy_score 0.9996289424860854
recall_score 0.9991645781119465
precision_score 0.998330550918197
f1_score 0.9987473903966597
======Training stage======
Epoch 1/30
10396/10396 [==============================] - 3s 242us/step - loss: 5.3347e-04 - accuracy: 0.9999
Epoch 2/30
10396/10396 [==============================] - 3s 263us/step - loss: 5.0691e-04 - accuracy: 0.9999
Epoch 3/30
10396/10396 [==============================] - 2s 235us/step - loss: 4.6739e-04 - accuracy: 0.9999
Epoch 4/30
10396/10396 [==============================] - 2s 234us/step - loss: 4.7805e-04 - accuracy: 0.9999
Epoch 5/30
10396/10396 [==============================] - 2s 231us/step - loss: 4.2914e-04 - accuracy: 0.9999
Epoch 6/30
10396/10396 [==============================] - 2s 232us/step - loss: 4.9170e-04 - accuracy: 0.9999
Epoch 7/30
10396/10396 [==============================] - 2s 229us/step - loss: 4.5416e-04 - accuracy: 0.9999
Epoch 8/30
10396/10396 [==============================] - 2s 231us/step - l

accuracy_score 0.9995052566481137
recall_score 0.9968178202068417
precision_score 1.0
f1_score 0.9984063745019921

K-fold cross validation Accuracy: [0.9922145328719724, 0.9956709956709957, 0.9965367965367965, 0.9992578849721707, 0.9985157699443413, 1.0, 1.0, 0.9996289424860854, 0.9996289424860854, 0.9995052566481137]

K-fold cross validation Recall: [0.9648, 0.9807370184254607, 0.9913928012519562, 0.9974662162162162, 0.9967715899919289, 1.0, 1.0, 0.9974048442906575, 0.9991645781119465, 0.9968178202068417]

K-fold cross validation Precision: [0.9844897959183674, 0.9898562975486053, 0.9867601246105919, 0.9974662162162162, 0.9935639581657281, 1.0, 1.0, 1.0, 0.998330550918197, 1.0]

K-fold cross validation F1-Score: [0.9745454545454545, 0.985275557425326, 0.9890710382513661, 0.9974662162162162, 0.9951651893634165, 1.0, 1.0, 0.9987007362494587, 0.9987473903966597, 0.9984063745019921]


In [10]:
# 4gram 평가지표
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.998095912161656
10-Fold Cross_validation. Recall : 0.9924554868495006
10-Fold Cross_validation. Precision : 0.9950466943377705
10-Fold Cross_validation. F1-Score : 0.993737795694989
